# Extração de dados

## Contexto:

A  Physiotherapy Evidence Database (PEDro) foi utlizada, pois é uma base de dados especificamente para artigos de fisioterapia.
A motivação para o projeto foi a minha propria experiencia profissional e as discussões com colegas de profissão, levantando a problematica do nível de qualidade das evidencias científicas para fundamentar as condutas terapêuticas, pois não há possibilidade de entregar um bom atendimento sem fundamenta-lo em informações confiáveis e de qualidade.

Esse notebook é a parte inicial para poder montar a base de dados utilizada nas análises, que estão apresentadas no outro notebook do repositório.

## Objetivo:
- Detectar artigos com o tema de estudo. Para esse exemplo foi escolhido a acupuntura
- Extrair informações relevantes sobre os artigos encontrados e armazena-los de maneira adequada

# 1.Carregando bibliotecas

In [3]:
import pandas as pd
import re

from queue import Full
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager

# 2.Inicializando o driver do chrome e acessando o endereço

- Podemos escolher entre visualizar todo o processo ou esconder o navegador

Opção com visualização do processo no navegador

In [4]:
# driver = webdriver.Chrome()
from webdriver_manager.core.utils import ChromeType
driver = webdriver.Chrome(service=Service(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
driver.maximize_window()
driver.get("https://pedro.org.au/")
# O método sleep é utilizado aqui para permitir o carregamento da página, evitando a quebra do código durante a execução
sleep(2)

"Escondendo navegador"

In [5]:
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)
# driver.get("https://pedro.org.au/")
# sleep(2)

Criando a função de busca nos campos

In [6]:
def caixa_busca(nome, cod):
    if cod != "":
        driver.find_element(By.ID, nome).send_keys(cod)
    else:
        pass

# 3.Interagindo com a página e escolhendo parametros para busca

## 3.1.Identificando botão para a página de busca

In [7]:
botao0 = driver.find_element(By.XPATH, "/html/body/header/div/nav[2]/div/div/ul/li[1]/a")
botao0.click()
sleep(2)

## 3.2.Enviando as informações para busca dos artigos

In [8]:
# Campo para especificar o titulo/resumo (livre)
# caixa_busca('abstract_with_title', '')
caixa_busca('abstract_with_title', 'acupuncture')

# -----------------------------------------------------------------------------------------
# Campo para especificar a região do corpo (lista predeterminada)
# regiao = [
#    'head or neck', 'upper arm, shoulder or shoulder gridle', 'forearm or elbow',
#    'hand or wrist', 'chest', 'thoracic spine', 'lumbar spine, sacro-iliac joint or pelvis',
#    'perinium or genito-urinary system', 'thigh or hip', 'lower leg or knee', 'foot or ankle'
#    ]
# caixa_busca('body_part', 'upper arm, shoulder or shoulder gridle')

# -----------------------------------------------------------------------------------------
# Campo para especificar o metodo do artigo (lista predeterminada)
lista_metodo = ['practice guideline', 'systematic review', 'clinical trial']
# met = lista_metodo[]
met = lista_metodo[2]
caixa_busca('method', '')

# -----------------------------------------------------------------------------------------
# Campo para especificar desde que ano queremos publicações (livre)
# caixa_busca('year_of_publication', '')
caixa_busca('year_of_publication', '')

# -----------------------------------------------------------------------------------------
# Campo para especificar a pontuação minima na escala PEDro (de 1 a 10)
# caixa_busca('nscore', '')
caixa_busca('nscore', '')

# -----------------------------------------------------------------------------------------
# Campo para especificar a quantidade de artigos por pagina (lista predeterminada porém
# estabeleci em 50, pois assim é necessário menos interações com a pagina)
# caixa_busca('perpage', '')
caixa_busca('perpage', '50')

# -----------------------------------------------------------------------------------------
# Botão para iniciar a busca
botao1 = driver.find_element(By.CLASS_NAME, "button")
botao1.click()
sleep(2)

# 4.Passando pelas paginas de resultados e armazenando dados iniciais

In [9]:
proxima_pagina = driver.find_elements(By.PARTIAL_LINK_TEXT, '»')
lista_geral_titulo = []
lista_geral_links = []
lista_scihub = []

try:
    while proxima_pagina != False:
        lista_pagina = driver.find_elements(By.CLASS_NAME, "left")
        for titulo in lista_pagina:
            lista_geral_titulo.append(titulo.text)
            lista_geral_links.append(titulo.get_attribute('href'))
        sleep(1)
        proxima_pagina = driver.find_element(By.PARTIAL_LINK_TEXT, '»')
        proxima_pagina.click()
except:
    print("")
    print("="*28)
    print("{} artigo(s) encontrado(s)".format(len(lista_geral_titulo)))
    print("="*28)


3765 artigo(s) encontrado(s)


# 5.Extraindo informações dos artigos encontrados

In [9]:
n = len(lista_geral_titulo)
selecao_artigos = [i for i in range(1, n+1)]
lista_revistas = []
lista_tipos = []
lista_escalas = []

for num in selecao_artigos:
    num = int(num)
    num -= 1
    driver.get(lista_geral_links[num])
    sleep(2)
    revista = driver.find_element(By.XPATH, '//*[@id="search-content"]/table/tbody/tr[3]/td')
    tipo = driver.find_element(By.XPATH, '//*[@id="search-content"]/table/tbody/tr[4]/td')
    try:
        escala_pedro = driver.find_element(By.XPATH, '//*[@id="search-content"]/table/tbody/tr[5]')
    except:
        escala_pedro = driver.find_element(By.XPATH, '//*[@id="search-content"]/table/tbody/tr[5]/td')
    finally:
        pass
    lista_revistas.append(revista.text)
    lista_tipos.append(tipo.text.upper())
    if met == lista_metodo[2]:
        try:
            lista_escalas.append(escala_pedro.text[0])
        except:
            lista_escalas.append('-')
            pass
    else:
        pass

# 6.Criando dataframe sobre o tema pesquisado

In [13]:
df_acupuntura = pd.DataFrame([lista_geral_titulo, lista_escalas, lista_revistas, lista_tipos])
df_acupuntura = df_acupuntura.T
df_acupuntura.columns = ['titulo', 'escala pedro', 'revista', 'tipo estudo']


# 7. Extraindo o ano de publicação 

In [14]:
lista_ano = []

for item in df_acupuntura['revista']:
    ano = re.search(r'\d{4}', item)
    lista_ano.append(int(ano.group()))

# 8.Criando dataframe final e salvando

In [15]:
df_acupuntura = pd.concat([df_acupuntura, pd.DataFrame(lista_ano)], axis=1)

In [16]:
df_acupuntura.columns = ['titulo', 'escala pedro', 'revista', 'tipo estudo', 'ano']

In [17]:
df_acupuntura.to_csv(r'data/acupuntura.csv')